## Pub Med Scraper

In [2]:
# scrape dependencies
import requests
import re
from bs4 import BeautifulSoup as bs

# data analysis dependencies
import pandas as pd
import numpy as np
import csv

# ipynb dependencies
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

# viz dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

import datetime as dt
import time

In [3]:
# set the url to scrape
url = 'https://www.ncbi.nlm.nih.gov/pubmed/trending/'
print(url)

https://www.ncbi.nlm.nih.gov/pubmed/trending/


In [4]:
# set up beautiful soup to scrape
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [5]:
# lets scrape the article titles
journals = soup.find_all("p", attrs={'class':'title'})

In [6]:
# searching for the journal titles
journals_len = len(journals)
print(f"There are {journals_len} journals to scrape on the first page.")

There are 20 journals to scrape on the first page.


In [7]:
# loop through journals to print titles
for i in range(journals_len):
    journals[i].text.strip()

'High-quality genomes reveal new differences between the great apes.'

'The role of ADAMTS13 testing in the diagnosis and management of thrombotic microangiopathies and thrombosis.'

'Reprogramming human T cell function and specificity with non-viral genome targeting.'

'The oncometabolite D-2-hydroxyglutarate is an intercellular mediator in IDH-mutant gliomas that inhibits both complement and T cells.'

'Unexpected similarities between C9ORF72 and sporadic forms of ALS/FTD suggest a common disease mechanism.'

...

'Günter Blobel (1936–2018)'

'Bacterial Endospores as Phage Genome Carriers and Protective Shells.'

'Histidine catabolism is a major determinant of methotrexate sensitivity.'

'Enhancing T cell therapy through TCR-signaling-responsive nanoparticle drug delivery.'

'Health risks of using mothballs in Greater Accra, Ghana.'

## Set main url to concat with pubmed ids

In [8]:
# set the main url that we will concatanate with the pubmed id
main_url = 'https://www.ncbi.nlm.nih.gov/pubmed/'
print(main_url)

https://www.ncbi.nlm.nih.gov/pubmed/


<br>
<br>
# TESTING SECTION ------------------------------------------------------------------

In [9]:
# set empty links_all list to append to 
links_all = []

# set pubmed ids list to append to
pubmed_ids = []

# set empty list to append scrape_links to
scrape_links = []

In [10]:
# use bs to scarpe p tags with class - title
links = soup.find_all("p",attrs={'class':'title'})
print(links[0])

<p class="title" xmlns:mml="http://www.w3.org/1998/Math/MathML"><a href="/pubmed/30006623" ref="ordinalpos=1&amp;ncbi_uid=30006623&amp;link_uid=30006623&amp;linksrc=docsum_title">High-quality genomes reveal new differences between the great apes.</a></p>


In [11]:
# testing to see how my links / journals to scrape
articles_to_scrape = len(links)
print(f"There are {articles_to_scrape} articles to scrape.")
print("----------------------------------------------")

There are 20 articles to scrape.
----------------------------------------------


In [12]:
# loop through links to convert to string
for i in range (len(links)):
    links_all.append(str(links[i]))
    print(links[i])
    print("----------------------------------------------")

<p class="title" xmlns:mml="http://www.w3.org/1998/Math/MathML"><a href="/pubmed/30006623" ref="ordinalpos=1&amp;ncbi_uid=30006623&amp;link_uid=30006623&amp;linksrc=docsum_title">High-quality genomes reveal new differences between the great apes.</a></p>
----------------------------------------------
<p class="title" xmlns:mml="http://www.w3.org/1998/Math/MathML"><a href="/pubmed/30006329" ref="ordinalpos=2&amp;ncbi_uid=30006329&amp;link_uid=30006329&amp;linksrc=docsum_title">The role of ADAMTS13 testing in the diagnosis and management of thrombotic microangiopathies and thrombosis.</a></p>
----------------------------------------------
<p class="title" xmlns:mml="http://www.w3.org/1998/Math/MathML"><a href="/pubmed/29995861" ref="ordinalpos=3&amp;ncbi_uid=29995861&amp;link_uid=29995861&amp;linksrc=docsum_title">Reprogramming human T cell function and specificity with non-viral genome targeting.</a></p>
----------------------------------------------
<p class="title" xmlns:mml="http://w

In [13]:
# for i in links_all:
#     print(i)
#     print("----------------------------------------------")

In [14]:
# slice through links_all to test
len(links_all)
links_all[0]

20

'<p class="title" xmlns:mml="http://www.w3.org/1998/Math/MathML"><a href="/pubmed/30006623" ref="ordinalpos=1&amp;ncbi_uid=30006623&amp;link_uid=30006623&amp;linksrc=docsum_title">High-quality genomes reveal new differences between the great apes.</a></p>'

### Regex test

In [15]:
regex_test = '<p class="title" xmlns:mml="http://www.w3.org/1998/Math/MathML"><a href="/pubmed/29985486" ref="ordinalpos=1&amp;ncbi_uid=29985486&amp;link_uid=29985486&amp;linksrc=docsum_title">Author Correction: What models eat.</a></p>'

In [16]:
print(regex_test)

<p class="title" xmlns:mml="http://www.w3.org/1998/Math/MathML"><a href="/pubmed/29985486" ref="ordinalpos=1&amp;ncbi_uid=29985486&amp;link_uid=29985486&amp;linksrc=docsum_title">Author Correction: What models eat.</a></p>


In [17]:
re.findall(r'\d{8}',regex_test)

['29985486', '29985486', '29985486']

### End Regex Test

In [18]:
# loop through links all and use regex to grab the id numbers
for i in range (len(links_all)):
    pubmed_ids.append(re.findall(r'\d{8}',links_all[i]))

In [19]:
# print out info for pubmed_ids
len(pubmed_ids)
type(pubmed_ids)
print(pubmed_ids)
print("----------------------------------------------")

20

list

[['30006623', '30006623', '30006623'], ['30006329', '30006329', '30006329'], ['29995861', '29995861', '29995861'], ['30006485', '30006485', '30006485'], ['30003873', '30003873', '30003873'], ['30010673', '30010673', '30010673'], ['30006565', '30006565', '30006565'], ['30001199', '30001199', '30001199'], ['30006921', '30006921', '30006921'], ['29985486', '29985486', '29985486'], ['29988082', '29988082', '29988082'], ['30006739', '30006739', '30006739'], ['30007343', '30007343', '30007343'], ['30006038', '30006038', '30006038'], ['29977318', '29977318', '29977318'], ['29975496', '29975496', '29975496'], ['30006404', '30006404', '30006404'], ['29995852', '29995852', '29995852'], ['29985479', '29985479', '29985479'], ['21967193', '21967193', '21967193']]
----------------------------------------------


In [20]:
# use itertools to transform pubmed ids from an array withn an array into one list
import itertools
pubmed_merged = list(itertools.chain.from_iterable(pubmed_ids))

In [21]:
# slice through pubmed_merged to see what itertools did
pubmed_merged[0]
print(pubmed_merged)

'30006623'

['30006623', '30006623', '30006623', '30006329', '30006329', '30006329', '29995861', '29995861', '29995861', '30006485', '30006485', '30006485', '30003873', '30003873', '30003873', '30010673', '30010673', '30010673', '30006565', '30006565', '30006565', '30001199', '30001199', '30001199', '30006921', '30006921', '30006921', '29985486', '29985486', '29985486', '29988082', '29988082', '29988082', '30006739', '30006739', '30006739', '30007343', '30007343', '30007343', '30006038', '30006038', '30006038', '29977318', '29977318', '29977318', '29975496', '29975496', '29975496', '30006404', '30006404', '30006404', '29995852', '29995852', '29995852', '29985479', '29985479', '29985479', '21967193', '21967193', '21967193']


In [22]:
# concat main_url with a slice of pubmed_merged before we loop
print(main_url + str(pubmed_merged[0]))

https://www.ncbi.nlm.nih.gov/pubmed/30006623


In [23]:
# append merged links to links_all
for i in range (len(pubmed_merged)):
    scrape_links.append(main_url + str(pubmed_merged[i]))

In [24]:
for i in scrape_links:
    print(i)

https://www.ncbi.nlm.nih.gov/pubmed/30006623
https://www.ncbi.nlm.nih.gov/pubmed/30006623
https://www.ncbi.nlm.nih.gov/pubmed/30006623
https://www.ncbi.nlm.nih.gov/pubmed/30006329
https://www.ncbi.nlm.nih.gov/pubmed/30006329
https://www.ncbi.nlm.nih.gov/pubmed/30006329
https://www.ncbi.nlm.nih.gov/pubmed/29995861
https://www.ncbi.nlm.nih.gov/pubmed/29995861
https://www.ncbi.nlm.nih.gov/pubmed/29995861
https://www.ncbi.nlm.nih.gov/pubmed/30006485
https://www.ncbi.nlm.nih.gov/pubmed/30006485
https://www.ncbi.nlm.nih.gov/pubmed/30006485
https://www.ncbi.nlm.nih.gov/pubmed/30003873
https://www.ncbi.nlm.nih.gov/pubmed/30003873
https://www.ncbi.nlm.nih.gov/pubmed/30003873
https://www.ncbi.nlm.nih.gov/pubmed/30010673
https://www.ncbi.nlm.nih.gov/pubmed/30010673
https://www.ncbi.nlm.nih.gov/pubmed/30010673
https://www.ncbi.nlm.nih.gov/pubmed/30006565
https://www.ncbi.nlm.nih.gov/pubmed/30006565
https://www.ncbi.nlm.nih.gov/pubmed/30006565
https://www.ncbi.nlm.nih.gov/pubmed/30001199
https://ww

# END TESTING SECTION------------------------------------------------------------
<br>
<br>

## Function to create array of links to scrape

In [25]:
# set empty links_all list to append to 
links_all = []

# set pubmed ids list to append to
pubmed_ids = []

# set empty list to append scrape_links to
scrape_links = []

# function to get links
def get_links(main_url):
    
    # use bs to scarpe p tags with class - title
    links = soup.find_all("p",attrs={'class':'title'})
      
    # testing to see how my links / journals to scrape
    articles_to_scrape = len(links)
    print(f"There are {articles_to_scrape} articles to scrape.")
    print("----------------------------------------------")
    
    # loop through links to convert to string
    for i in range (len(links)):
        links_all.append(str(links[i]))
        print(links[i])
        print("----------------------------------------------")
        
    # slice through links_all to test
    len(links_all)
    links_all[1]
    
    # loop through links all and use regex to grab the id numbers
    for i in range (len(links_all)):
        pubmed_ids.append(re.findall(r'\d{8}',links_all[i]))
    
    # print out info for pubmed_ids
    len(pubmed_ids)
    type(pubmed_ids)
    print(pubmed_ids)
    print("----------------------------------------------")
    
    # use itertools to transform pubmed ids from an array withn an array into one list
    import itertools
    pubmed_merged = list(itertools.chain.from_iterable(pubmed_ids))
    
    # slice through pubmed_merged to see what itertools did
    pubmed_merged[0]
    
    # concat main_url with a slice of pubmed_merged before we loop
    print(main_url + str(pubmed_merged[0]))
    
    # append merged links to links_all
    for i in range (len(pubmed_merged)):
        scrape_links.append(main_url + str(pubmed_merged[i]))

In [26]:
# RUN FUNCTION
get_links(main_url)

There are 20 articles to scrape.
----------------------------------------------
<p class="title" xmlns:mml="http://www.w3.org/1998/Math/MathML"><a href="/pubmed/30006623" ref="ordinalpos=1&amp;ncbi_uid=30006623&amp;link_uid=30006623&amp;linksrc=docsum_title">High-quality genomes reveal new differences between the great apes.</a></p>
----------------------------------------------
<p class="title" xmlns:mml="http://www.w3.org/1998/Math/MathML"><a href="/pubmed/30006329" ref="ordinalpos=2&amp;ncbi_uid=30006329&amp;link_uid=30006329&amp;linksrc=docsum_title">The role of ADAMTS13 testing in the diagnosis and management of thrombotic microangiopathies and thrombosis.</a></p>
----------------------------------------------
<p class="title" xmlns:mml="http://www.w3.org/1998/Math/MathML"><a href="/pubmed/29995861" ref="ordinalpos=3&amp;ncbi_uid=29995861&amp;link_uid=29995861&amp;linksrc=docsum_title">Reprogramming human T cell function and specificity with non-viral genome targeting.</a></p>
---

<br>
There are duplicates in our **scrape_links** array. Use `list` to delete the duplicates.

In [27]:
# delete duplicates in scrape_links and assign to new variable scrape_links_final
scrape_links_final = list(set(scrape_links))
len(scrape_links_final)
scrape_links_final

20

['https://www.ncbi.nlm.nih.gov/pubmed/30006623',
 'https://www.ncbi.nlm.nih.gov/pubmed/30006404',
 'https://www.ncbi.nlm.nih.gov/pubmed/30006739',
 'https://www.ncbi.nlm.nih.gov/pubmed/29977318',
 'https://www.ncbi.nlm.nih.gov/pubmed/29985479',
 'https://www.ncbi.nlm.nih.gov/pubmed/30006921',
 'https://www.ncbi.nlm.nih.gov/pubmed/21967193',
 'https://www.ncbi.nlm.nih.gov/pubmed/30010673',
 'https://www.ncbi.nlm.nih.gov/pubmed/29995861',
 'https://www.ncbi.nlm.nih.gov/pubmed/30006329',
 'https://www.ncbi.nlm.nih.gov/pubmed/30003873',
 'https://www.ncbi.nlm.nih.gov/pubmed/29995852',
 'https://www.ncbi.nlm.nih.gov/pubmed/30007343',
 'https://www.ncbi.nlm.nih.gov/pubmed/30006485',
 'https://www.ncbi.nlm.nih.gov/pubmed/29985486',
 'https://www.ncbi.nlm.nih.gov/pubmed/29988082',
 'https://www.ncbi.nlm.nih.gov/pubmed/30001199',
 'https://www.ncbi.nlm.nih.gov/pubmed/30006038',
 'https://www.ncbi.nlm.nih.gov/pubmed/30006565',
 'https://www.ncbi.nlm.nih.gov/pubmed/29975496']

## Main array of links to scrape:

Here we use selenium to iterate through these links. Seleium will click on each link then scrape the title and abstracts on each page. 

In [28]:
# testing scrape_links
for i in scrape_links_final:
    print(i)

https://www.ncbi.nlm.nih.gov/pubmed/30006623
https://www.ncbi.nlm.nih.gov/pubmed/30006404
https://www.ncbi.nlm.nih.gov/pubmed/30006739
https://www.ncbi.nlm.nih.gov/pubmed/29977318
https://www.ncbi.nlm.nih.gov/pubmed/29985479
https://www.ncbi.nlm.nih.gov/pubmed/30006921
https://www.ncbi.nlm.nih.gov/pubmed/21967193
https://www.ncbi.nlm.nih.gov/pubmed/30010673
https://www.ncbi.nlm.nih.gov/pubmed/29995861
https://www.ncbi.nlm.nih.gov/pubmed/30006329
https://www.ncbi.nlm.nih.gov/pubmed/30003873
https://www.ncbi.nlm.nih.gov/pubmed/29995852
https://www.ncbi.nlm.nih.gov/pubmed/30007343
https://www.ncbi.nlm.nih.gov/pubmed/30006485
https://www.ncbi.nlm.nih.gov/pubmed/29985486
https://www.ncbi.nlm.nih.gov/pubmed/29988082
https://www.ncbi.nlm.nih.gov/pubmed/30001199
https://www.ncbi.nlm.nih.gov/pubmed/30006038
https://www.ncbi.nlm.nih.gov/pubmed/30006565
https://www.ncbi.nlm.nih.gov/pubmed/29975496


### Add timer to main function when scraping

In [41]:
# slice out scrape_links_final so we can scrape 5 articles at a time
links_1 = scrape_links_final[0:1]
links_1

links_2 = scrape_links_final[5:6]
links_2

links_3 = scrape_links_final[10:15]
links_3

links_4 = scrape_links_final[15:20]
links_4

['https://www.ncbi.nlm.nih.gov/pubmed/30006623']

['https://www.ncbi.nlm.nih.gov/pubmed/30006921']

['https://www.ncbi.nlm.nih.gov/pubmed/30003873',
 'https://www.ncbi.nlm.nih.gov/pubmed/29995852',
 'https://www.ncbi.nlm.nih.gov/pubmed/30007343',
 'https://www.ncbi.nlm.nih.gov/pubmed/30006485',
 'https://www.ncbi.nlm.nih.gov/pubmed/29985486']

['https://www.ncbi.nlm.nih.gov/pubmed/29988082',
 'https://www.ncbi.nlm.nih.gov/pubmed/30001199',
 'https://www.ncbi.nlm.nih.gov/pubmed/30006038',
 'https://www.ncbi.nlm.nih.gov/pubmed/30006565',
 'https://www.ncbi.nlm.nih.gov/pubmed/29975496']

## Regex Notes

In [30]:
# Regex
# Identifiers:
# \d any number
# \D anything but a number
# \s space
# \S anything but a space
# \w any character
# \W anything but a character
# . any character, except for a newline
# \b the whitespace around words
# \. a period

# Modifiers:
# {1,3} we're expecting 1-3 \d{1-3}
# + Match 1 or more
# ? Match 0 or more
# * Match 0 or more
# $ Match the end of a string
# ^ matching the beginning of a string
# | either or
# [] range or "variance" [A-Za-z] [1-5a-qA-Z]
# {x} expecting "x" amount

# White Space Characters: 
# \n new line
# \s space
# \t tab
# \e escape
# \f form feed
# \r return

# DONT FORGET!:
# . + * [] $ ^ () {} | \

## Selenium
**Web Browser Automation**

In [31]:
from splinter import Browser
from selenium import webdriver

In [32]:
# make sure chrome browser exe is in current directory
# chrome browser exe is not necessary for MACS
executable_path = {'executable_path': 'chromedriver'}

## Set up dictionary to append data to

In [33]:
article_dict = {}

## Create get_article_info function

In [43]:
title = []
abstract = []
authors = []

for i in links_1:
    
        browser = Browser('chrome', headless=False)
        html = browser.html
        response2 = requests.get(i)
        soup2 = bs(response2.text, 'html.parser')
    
        browser.visit(i)
    
        # there are two 'h1' tags on this page. slice out index 0
        title_one = soup2.find_all('h1')
        article_one_title = title_one[1].text.strip()
    
        # slice h1 at index 1 to grab article title
        title.append(article_one_title)
    
        # get abstract 
        # abstract.append(soup2.find("div", attrs={'class': 'rprt_all'}).text.strip())     
        abstract.append(soup2.find("div", attrs={'class': 'abstr'}))
        
        # get authors
        #for data in soup2.find('div', attrs={'class': 'auths'}):
        #    for a in data.find_all('a'):
        #        print(a.text)
        #        print(a.get('href'))
        
        article_dict["title"] = title
        article_dict["abstract"] = abstract
        #article_dict["authors"] = authors
        

In [37]:
title

['High-quality genomes reveal new differences between the great apes.']

# strip article for author, abstract, other info

In [45]:
# sets up scraper
from splinter import Browser


from selenium import webdriver

# make sure chrome browser exe is in current directory
# chrome browser exe is not necessary for MACS
executable_path = {'executable_path': 'chromedriver'}

title = []
abstract = []
authors = []

url2 = links_2

browser = Browser('chrome', headless=False)
html = browser.html
response2 = requests.get(url2)
soup2 = bs(response2.text, 'html.parser')
    
browser.visit(url2)
  

InvalidSchema: No connection adapters were found for '['https://www.ncbi.nlm.nih.gov/pubmed/30006921']'

In [ ]:
  
# there are two 'h1' tags on this page. slice out index 0
title_one = soup2.find_all('h1')
article_one_title = title_one[1].text.strip()
    
# slice h1 at index 1 to grab article title
title.append(article_one_title)
    
# get abstract 
# abstract.append(soup2.find("div", attrs={'class': 'rprt_all'}).text.strip())     
abstract.append(soup2.find("div", attrs={'class': 'rprt abstract'}).text.strip())
        
# get authors
authors.append(soup2.find("div", attrs={'class': 'auths'}).text.strip())
        


## Run article_info function

In [35]:
get_article_info(links_1)

In [ ]:
get_article_info(links_2)

In [ ]:
get_article_info(links_3)

In [ ]:
get_article_info(links_4)

In [36]:
for i in title:
    print(i + "\n")

len(title)

DNMT3A and TET1 cooperate to regulate promoter epigenetic landscapes in mouse embryonic stem cells.

Health risks of using mothballs in Greater Accra, Ghana.

Author Correction: What models eat.

The role of ADAMTS13 testing in the diagnosis and management of thrombotic microangiopathies and thrombosis.

Reprogramming human T cell function and specificity with non-viral genome targeting.



5

In [37]:
for i in abstract:
    print(i)
    print("\n")

len(abstract)

Genome Biol. 2018 Jul 12;19(1):88. doi: 10.1186/s13059-018-1464-7.DNMT3A and TET1 cooperate to regulate promoter epigenetic landscapes in mouse embryonic stem cells.Gu T1, Lin X2,3,4,5, Cullen SM1,6,7, Luo M1, Jeong M1, Estecio M8, Shen J8, Hardikar S8, Sun D2,3,9, Su J2,3, Rux D10, Guzman A1, Lee M9, Qi LS5, Chen JJ11, Kyba M10, Huang Y9, Chen T8, Li W12,13, Goodell MA14,15.Author information1Stem Cells and Regenerative Medicine Center, Baylor College of Medicine, Houston, TX, 77030, USA.2Division of Biostatistics, Dan L. Duncan Cancer Center, Baylor College of Medicine, Houston, TX, 77030, USA.3Department of Molecular and Cellular Biology, Baylor College of Medicine, Houston, TX, 77030, USA.4Department of Bioinformatics, School of Life Sciences and Technology, Tongji University, Shanghai, China.5Department of Bioengineering, Stanford University, Stanford, California, USA.6Program in Developmental Biology, Baylor College of Medicine, Houston, TX, 77030, USA.7Medical Scientist Training

5

## Add title and abstract to article_dict

In [38]:
# check article dict
article_dict

{'abstract': ["Genome Biol. 2018 Jul 12;19(1):88. doi: 10.1186/s13059-018-1464-7.DNMT3A and TET1 cooperate to regulate promoter epigenetic landscapes in mouse embryonic stem cells.Gu T1, Lin X2,3,4,5, Cullen SM1,6,7, Luo M1, Jeong M1, Estecio M8, Shen J8, Hardikar S8, Sun D2,3,9, Su J2,3, Rux D10, Guzman A1, Lee M9, Qi LS5, Chen JJ11, Kyba M10, Huang Y9, Chen T8, Li W12,13, Goodell MA14,15.Author information1Stem Cells and Regenerative Medicine Center, Baylor College of Medicine, Houston, TX, 77030, USA.2Division of Biostatistics, Dan L. Duncan Cancer Center, Baylor College of Medicine, Houston, TX, 77030, USA.3Department of Molecular and Cellular Biology, Baylor College of Medicine, Houston, TX, 77030, USA.4Department of Bioinformatics, School of Life Sciences and Technology, Tongji University, Shanghai, China.5Department of Bioengineering, Stanford University, Stanford, California, USA.6Program in Developmental Biology, Baylor College of Medicine, Houston, TX, 77030, USA.7Medical Sci

## Save article_dict to json

In [39]:
import json

In [40]:
# check json file
json = json.dumps(article_dict)
f = open("trending2.json","w")
f.write(json)
f.close()

22182

In [41]:
# Import sqlalchemy Dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
from sqlalchemy import Column, Float, Integer, String, Date
from sqlalchemy.ext.declarative import declarative_base

In [42]:
# Create an engine for the photopharm.sqlite database
engine = create_engine("sqlite:///photopharm.sqlite")


In [43]:
# Reflect Database into ORM classes
Base = declarative_base()

In [49]:
# Create ORM class for titles and abstracts
class Articles(Base):
    __tablename__ = "Title_Abstracts"
    
    __table_args__ = {'extend_existing': True} 
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    abstract = Column(String)
    

In [50]:
# create tables
Base.metadata.create_all(engine)

In [51]:
# read in trending1 json into a dataframe
df2 = pd.read_json('trending2.json')

In [52]:
df2

,abstract,title
0,Genome Biol. 2018 Jul 12;19(1):88. doi: 10.118...,DNMT3A and TET1 cooperate to regulate promoter...
1,Trop Med Int Health. 2012 Jan;17(1):135-8. doi...,Health risks of using mothballs in Greater Acc...
2,Nat Med. 2018 Jul 9. doi: 10.1038/s41591-018-0...,Author Correction: What models eat.
3,Blood. 2018 Jul 13. pii: blood-2018-02-791533....,The role of ADAMTS13 testing in the diagnosis ...
4,Nature. 2018 Jul 11. doi: 10.1038/s41586-018-0...,Reprogramming human T cell function and specif...


In [53]:
# import json to sqlite
df2.to_sql(con=engine, index_label='id', name=Articles.__tablename__, if_exists='append')